# Maximum inner product search using Asymmetric Locality Sensitive Hashing

## For small Dataset

In [6]:
import os
import pandas as pd
import numpy as np

dataset = "datasets"+os.path.sep+"ml-latest-small"
#dataset = "datasets"+os.path.sep+"ml-latest"
name = "ratings.csv"
ratings_df = pd.read_csv(dataset + os.path.sep + name, names= ["UserID", "MovieID", "Rating", "Timestamp"], header=0)
ratings_df["UserID"] = pd.to_numeric(ratings_df["UserID"], errors='ignore')
ratings_df["MovieID"] = pd.to_numeric(ratings_df["MovieID"], errors='ignore')
ratings_df["Rating"] = pd.to_numeric(ratings_df["Rating"], errors='ignore')
ratings_df["Timestamp"] = pd.to_numeric(ratings_df["Timestamp"], errors='ignore')
R_df = ratings_df.pivot(index = 'UserID', columns ='MovieID', values = 'Rating').fillna(0)

R = R_df.as_matrix()
user_ratings_mean = np.mean(R, axis = 1)
R_demeaned = R - user_ratings_mean.reshape(-1, 1)


Executed on a smaller dataset at present


## Netflix Dataset

In [7]:
import pandas as pd
import numpy as np


users = []
movies_=[]
for p_ in range(0, 17770):
    movies_.append(p_)
    
    

values = np.zeros((480189, 17770), dtype='float32')
df = pd.DataFrame(data=values)

for x in range(0, 17770):
    name = "%07d" % (x+1,)
    name = (str)(name)
    name = "training_set/mv_"+name+".txt"
    f = open(name, "r")
    p = f.read()
    lines = p.split('\n')
    movie_=lines.pop(0)
    movie_=(int)(movie_[0:-1])
    if (movie_==x+1):
        print(x+1)
    for l in lines:
        if (l != ''):
            line1=l.split(',')
            print(line1[0])
            if (int)(line1[0]) not in users:
                users.append((int)(line1[0]))
        
            df[x][users.index((int)(line1[0]))] = (float)(line1[1])
    f.close()

    

    
R_df = df.pivot(index = users, columns = movies_).fillna(0)
R = R_df.as_matrix()
user_ratings_mean = np.mean(R, axis = 1)
R_demeaned = R - user_ratings_mean.reshape(-1, 1)
from scipy import io
io.mmwrite("Ratings_ML", R_demeaned, field='real', precision=4)


## Movielens Dataset

In [9]:
import os
import numpy as np
import pandas as pd
from scipy import sparse

ratings_df = pd.read_csv("ratings.dat",sep="::", names= ["UserID", "MovieID", "Rating", "Timestamp"], header=0)
ratings_df["UserID"] = pd.to_numeric(ratings_df["UserID"], errors='ignore')
ratings_df["MovieID"] = pd.to_numeric(ratings_df["MovieID"], errors='ignore')
ratings_df["Rating"] = pd.to_numeric(ratings_df["Rating"], errors='ignore')
ratings_df["Timestamp"] = pd.to_numeric(ratings_df["Timestamp"], errors='ignore')
R_df = ratings_df.pivot(index = 'UserID', columns ='MovieID', values = 'Rating').fillna(0)
R = R_df.as_matrix()
user_ratings_mean = np.mean(R, axis = 1)
R_demeaned = R - user_ratings_mean.reshape(-1, 1)
from scipy import io
io.mmwrite("Ratings_ML", R_demeaned, field='real', precision=4)


(100004, 4)

MovieID,1,2,3,4,5,6,7,8,9,10,...,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
UserID,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Test Train split



## 20% of the ratings are converted to zeroes in train data
## That 20% is our train data


###Implement cross-validation

In [13]:
from scipy import sparse

sR = sparse.csr_matrix(R_demeaned)
TRAIN_SIZE = 0.80
# Create boolean mask
# np.random creates a vector of random values between 0 and 1
# Those values are filtered to create a binary mask
msk = np.random.rand(sR.shape[0],sR.shape[1]) < TRAIN_SIZE
r = np.zeros(sR.shape)
print (msk.shape)
 
train_ratings = sR.copy()
test_ratings = sR.copy()
#mask itself is random
train_ratings[msk] = r[msk]
test_ratings[~msk] = r[~msk] # inverse of boolean mask
test_r = np.array(test_ratings)
train_r = np.array(train_ratings)

from scipy.sparse.linalg import svds
U, sigma, Vt = svds(R_demeaned, k = 50)


(671, 9066)


In [ ]:
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(R_demeaned, k = 50)
sigma = np.diag(sigma)


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
